In [1]:
import pandas as pd 
import numpy as np 

In [2]:
# Open the data # 
spike_times=pd.read_pickle('./klusters0/CR20B_120531_001_behaving.kkp') 
trials_info = pd.read_table('trials_info', sep=',', index_col=0) 
trial_timings=pd.read_table('trial_timings', sep=',', index_col=0)

In [3]:
# Following function are used to correct floating point of spike times
#Returns floats with defined precision 
def passing(N):
    l=[]
    for item in N:
        x=item*1000/1000
        y="%.3f" % x
        z=float(y)
        l.append
        
        
    return l

#Returns decimals 
def dec(N):
    l=[]
    
    from decimal import Context
    
    for item in N:
        context=Context(prec=3)
        x=context.create_decimal_from_float(item)
        l.append(x)
    return l

In [10]:
# Get spike and trial times , pass through smmoothing function and synchronize events: 

S=spike_times[spike_times["group"]==1]['time'].values.tolist()
Stim=trial_timings["stim_onset"].values.tolist()
Sp_1=np.array(passing(S))
Stim=np.array(passing(Stim))
# create time axis to bin spike train with 1ms accuracy
kl=np.arange(0,5000,0.001).tolist() 
time_axis=passing(kl)

# Synchronize events 

def Synchronize(x,y):
    List=[]
    
    for item in y:
        
        if item in x:
            List.append(1)
        else: 
            List.append(0)
            
    return List 


In [11]:
# More efficient way to align events 
S1=np.in1d(time_axis, Sp_1)

In [12]:
# Create a dataset with all events and spikes 
#DataTimeValues=pd.DataFrame({'G1':S1, 'G2':S2, 'G3':S3, 'G4':S4, 'G5':S5, 'G6':S6, 'G7':S7, 'G8':S8, 'Start':Start, 'Stop':Stop, 'Stim_time':Stim, 'Choice_time':Choice})
# Convert boolean to integer 
# DataTimeVal=DataTimeValues.astype(int)


In [14]:
# Get binned spike traines
#gp1=DataTimeVal['G1'].values.tolist()
#Get units from spike time  #example
u_1=spike_times[spike_times["group"]==1]['unit'].values.tolist()
x=np.unique(spike_times['unit'].values)
y=np.arange(1, len(np.unique(spike_times['unit'].values))+1)
d=dict(zip(np.unique(x), y))
#map dict to units 
u1=np.vectorize(d.get)(u_1).tolist()


In [15]:
# function to append units to the data 
def append_unit(x, y):  
    u=[]
    counter=0
    for i in x:
        if i==1:
            u.append(y[counter])
            counter+=1
        else: 
            u.append(0)
    return u
    

In [16]:
#unit1=append_unit(gp1, u1) Replace 1 with unit names 


In [17]:
def get_trials(x,y): #function that defines trials to train by trials
    v=[]
    counter=0
    for i in range(len(x)):
        if x[i]==1:
            counter+=1
            v.append(counter)
            for j in range(len(y)):
                if y[j]==0 and j >= i:
                    v.append(counter)
                    x[j]=2
                elif y[j]==1 and j> i:
                    break
        elif x[i]==0:
            v.append(0)
        else: 
            continue
    return v
            

In [18]:
#example 
#sp=DataTimeVal['Start'].values.tolist()
#dp=fp=DataTimeVal['Start'].values.tolist()
#h=get_trials(dp,fp)

In [19]:
#function to extract blocks
def get_blocks(x,y):
    block=[]
    for trial in x:
        if trial > 0:
            block.append(y[trial-1])
        else: block.append(0)
    return block
            

In [20]:
def oning(x,y):  #function that defines stimulation column to extract evoked activity 
    v=[]
    for i in range(len(x)):
        if x[i]==1:
            for j in range(len(y)):
                if y[j]==0 and j >= i:
                    v.append(1)
                    x[j]=2
                elif y[j]==1 and j> i:
                    break 
        elif x[i]==0:
            v.append(0)
        else: 
            continue
    return v

In [21]:
# function that enables to create dataset, where each column represents activity of separate unit, 
def get_neuron(S):
    df=pd.DataFrame(np.zeros(5000000))
    Number=np.unique(S).tolist()
        
    for number in Number:
        if number>0:
            
            d=S[S==number].index.values
            m=np.ones(len(d))
            r=np.zeros(5000000)
            r[d]=m
            df=pd.concat([df,pd.DataFrame(r)], axis=1)
    return df
            

In [22]:
#example
#f1=get_neuron(fantasy['unit1'])



In [23]:
# get activity of defined units during pitch discrimination and localisation trials 
#evoked=df_ready[df_ready['Stim_time']==1]
#pitch_evoked=evoked[evoked['stim_name']=='le_lo_pc']
#loc_evoked=evoked[evoked['stim_name']=='le_lo_lc']

